In [1]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram

from LogicalQ.LogicalGeneral import LogicalCircuitGeneral
from LogicalQ.Library.QECCs import five_qubit_code, steane_code, four_qubit_code
from LogicalQ.NoiseModel import construct_noise_model_QuantinuumH1_1
from LogicalQ.Experiments import execute_circuits

%load_ext autoreload
%autoreload 2

## Demonstrating error correction

In [2]:
code = five_qubit_code
n = code['label'][0]

In [3]:
#Tests errors measuring in Z basis
outputs = []
for i in range(n+1):
    five_qubit_circ = LogicalCircuitGeneral(2, **code)
    five_qubit_circ.encode(0,1, initial_states=[0,1])
    if i < n: #Runs the last sim without an error for thorough testing
        five_qubit_circ.add_error(0,i,'X') #Also try testing Z type errors
        five_qubit_circ.add_error(1,i,'X')
    five_qubit_circ.append_qec_cycle([0,1])
    five_qubit_circ.measure([0,1], [0,1], meas_basis='Z')
    simulator = AerSimulator()
    result = execute_circuits(five_qubit_circ, backend=simulator, shots=1, memory=True)[0]
    outputs.append(result.get_memory())

#The leftmost numbers in the output are the logical measurements. Should all be '10'
for o in outputs:
    print(o)

['10 00110 00 11 1000 1000 1000 0 00111 00 11 1000 1000 1000 0']
['10 10100 00 10 0001 0001 0001 0 11111 00 10 0001 0001 0001 0']
['10 11110 00 10 0011 0011 0011 0 01101 00 10 0011 0011 0011 0']
['10 00110 00 11 0110 0110 0110 0 11100 00 11 0110 0110 0110 0']
['10 10100 00 10 1100 1100 1100 0 11111 00 10 1100 1100 1100 0']
['10 11100 00 00 0000 0000 0000 0 01001 00 00 0000 0000 0000 0']


In [4]:
#Tests errors measuring in X basis
outputs = []
for i in range(n+1):
    five_qubit_circ = LogicalCircuitGeneral(2, **code)
    five_qubit_circ.encode(0,1, initial_states=[0,1])

    if i < n: #Runs the last sim without an error for thorough testing
        five_qubit_circ.add_error(0,i,'Z') #Also try testing Z type errors
        five_qubit_circ.add_error(1,i,'Z')
    five_qubit_circ.append_qec_cycle([0,1])

    five_qubit_circ.h([0,1])

    five_qubit_circ.measure([0,1], [0,1], meas_basis='X')
    
    simulator = AerSimulator()
    result = execute_circuits(five_qubit_circ, backend=simulator, shots=1, memory=True)[0]
    outputs.append(result.get_memory())

#The leftmost numbers in the output are the logical measurements. Should all be '10'
for o in outputs:
    print(o)

['10 10010 00 00 0101 0101 0101 0 00100 00 00 0101 0101 0101 0']
['10 00011 00 00 1010 1010 1010 0 00100 00 00 1010 1010 1010 0']
['10 00111 00 00 0100 0100 0100 0 01011 00 00 0100 0100 0100 0']
['10 10010 00 00 1001 1001 1001 0 10001 00 00 1001 1001 1001 0']
['10 11011 00 10 0010 0010 0010 0 00100 00 10 0010 0010 0010 0']
['10 11001 00 00 0000 0000 0000 0 00010 00 00 0000 0000 0000 0']


In [5]:
#Tests errors measuring in Y basis
outputs = []
for i in range(n+1):
    five_qubit_circ = LogicalCircuitGeneral(2, **code)
    five_qubit_circ.encode(0,1, initial_states=[0,1])
    
    if i < n: #Runs the last sim without an error for thorough testing
        five_qubit_circ.add_error(0,i,'X') #Also try testing Z type errors
        five_qubit_circ.add_error(1,i,'X')
    five_qubit_circ.append_qec_cycle([0,1])

    five_qubit_circ.h([0,1])
    five_qubit_circ.s([0,1])
    
    five_qubit_circ.measure([0,1], [0,1], meas_basis='Y', with_error_correction=True)
    simulator = AerSimulator()
    result = execute_circuits(five_qubit_circ, backend=simulator, shots=1, memory=True)[0]
    outputs.append(result.get_memory())

#The leftmost numbers in the output are the logical measurements. Should all be '10'
for o in outputs:
    print(o)

['10 00001 00 10 1000 1000 1000 0 01010 00 10 1000 1000 1000 0']
['10 00111 00 01 0001 0001 0001 0 01010 00 01 0001 0001 0001 0']
['10 10100 00 01 0011 0011 0011 0 11000 00 01 0011 0011 0011 0']
['10 01000 00 10 0110 0110 0110 0 10110 00 10 0110 0110 0110 0']
['10 01001 00 01 1100 1100 1100 0 00010 00 01 1100 1100 1100 0']
['10 10110 00 00 0000 0000 0000 0 01000 00 00 0000 0000 0000 0']


In [6]:
#Tests CX pauli frame updates
outputs = []
for i in range(n+1):
    five_qubit_circ = LogicalCircuitGeneral(2, **code)
    five_qubit_circ.encode(0,1, initial_states=[1,1])

    if i < n: #Runs the last sim without an error for thorough testing
        five_qubit_circ.add_error(1,i,'X') #Also try testing Z type errors

    five_qubit_circ.append_qec_cycle([0,1])

    five_qubit_circ.cx(1,0)

    five_qubit_circ.measure([0,1], [0,1], meas_basis='Z', with_error_correction=True)
    simulator = AerSimulator(method='stabilizer')
    result = execute_circuits(five_qubit_circ, backend=simulator, shots=1, memory=True)[0]
    outputs.append(result.get_memory())

#The leftmost numbers in the output are the logical measurements. Should all be '10'
for o in outputs:
    print(o)

['10 01111 00 11 1000 1000 1000 0 11111 00 10 0000 0000 0000 0']
['10 01010 00 10 0001 0001 0001 0 11010 00 10 0000 0000 0000 0']
['10 01111 00 10 0011 0011 0011 0 00111 00 10 0000 0000 0000 0']
['10 11011 00 11 0110 0110 0110 0 11010 00 10 0000 0000 0000 0']
['10 01010 00 10 1100 1100 1100 0 00001 00 10 0000 0000 0000 0']
['10 10101 00 00 0000 0000 0000 0 00011 00 00 0000 0000 0000 0']


In [7]:
#Tests CZ pauli frame updates
outputs = []
for i in range(n+1):
    five_qubit_circ = LogicalCircuitGeneral(2, **code)
    five_qubit_circ.encode(0,1, initial_states=[0,0])

    five_qubit_circ.h(1)
    five_qubit_circ.cx(1,0)

    if i < n: #Runs the last sim without an error for thorough testing
        five_qubit_circ.add_error(1,i,'X') #Also try testing Z type errors

    five_qubit_circ.append_qec_cycle([0,1])

    five_qubit_circ.cz(1,0)
    five_qubit_circ.cx(1,0)
    five_qubit_circ.h(1)

    five_qubit_circ.measure([0,1], [0,1], meas_basis='Z', with_error_correction=True)
    simulator = AerSimulator(method='stabilizer')
    result = execute_circuits(five_qubit_circ, backend=simulator, shots=1, memory=True)[0]
    outputs.append(result.get_memory())

#The leftmost numbers in the output are the logical measurements. Should all be '10'
for o in outputs:
    print(o)

['10 10000 00 01 1000 1000 1000 0 00100 00 11 0000 0000 0000 0']
['10 10001 00 11 0001 0001 0001 0 00001 00 11 0000 0000 0000 0']
['10 10001 00 11 0011 0011 0011 0 01101 00 11 0000 0000 0000 0']
['10 11001 00 01 0110 0110 0110 0 01011 00 11 0000 0000 0000 0']
['10 01100 00 11 1100 1100 1100 0 00010 00 11 0000 0000 0000 0']
['10 10011 00 00 0000 0000 0000 0 00110 00 00 0000 0000 0000 0']


In [8]:
#Tests CY pauli frame updates
outputs = []
for i in range(n+1):
    five_qubit_circ = LogicalCircuitGeneral(2, **code)
    five_qubit_circ.encode(0,1, initial_states=[0,0])

    five_qubit_circ.h(1)
    
    if i < n: #Runs the last sim without an error for thorough testing
        five_qubit_circ.add_error(0,i,'X') #Also try testing Z type errors

    five_qubit_circ.append_qec_cycle([0,1])

    five_qubit_circ.cy(1,0)
    five_qubit_circ.s(1)

    five_qubit_circ.cx(1,0)
    five_qubit_circ.h(1)

    five_qubit_circ.measure([0,1], [0,1], meas_basis='Z', with_error_correction=True)
    simulator = AerSimulator(method='stabilizer')
    result = execute_circuits(five_qubit_circ, backend=simulator, shots=1, memory=True)[0]
    outputs.append(result.get_memory())

#The leftmost numbers in the output are the logical measurements. Should all be '10'
for o in outputs:
    print(o)

['10 01111 00 10 0000 0000 0000 0 01101 00 11 1000 1000 1000 0']
['10 00110 00 10 0000 0000 0000 0 10011 00 10 0001 0001 0001 0']
['10 10100 00 10 0000 0000 0000 0 10011 00 10 0011 0011 0011 0']
['10 11000 00 10 0000 0000 0000 0 11100 00 11 0110 0110 0110 0']
['10 01010 00 10 0000 0000 0000 0 00100 00 10 1100 1100 1100 0']
['10 10110 00 00 0000 0000 0000 0 11011 00 00 0000 0000 0000 0']


In [ ]:
#Tests T and T^dagger gate Pauli updates
outputs = []
for i in range(n+1):
    five_qubit_circ = LogicalCircuitGeneral(2, **code)
    five_qubit_circ.encode(0,1, initial_states=[0,1])
    five_qubit_circ.h([0,1])
    five_qubit_circ.t([0,1])

    if i < n: #Runs the last sim without an error for thorough testing
        five_qubit_circ.add_error(0,i,'X') #Also try testing Z type errors
        five_qubit_circ.add_error(1,i,'X')
    five_qubit_circ.append_qec_cycle([0,1])

    five_qubit_circ.s([0,1])
    five_qubit_circ.tdg([0,1])
    five_qubit_circ.sdg([0,1])

    five_qubit_circ.h([0,1])

    five_qubit_circ.measure([0,1], [0,1], meas_basis='Z')
    simulator = AerSimulator()
    result = execute_circuits(five_qubit_circ, backend=simulator, shots=1, memory=True)[0]
    outputs.append(result.get_memory())

#The leftmost numbers in the output are the logical measurements. Should all be '10'
for o in outputs:
    print(o)

['10 10001 00 11 1000 1000 1000 0 00001 00 11 1000 1000 1000 0']
['10 11111 00 01 0001 0001 0001 0 11000 00 01 0001 0001 0001 0']
['10 00001 00 01 0011 0011 0011 0 01010 00 01 0011 0011 0011 0']
['10 00110 00 11 0110 0110 0110 0 10110 00 11 0110 0110 0110 0']
['10 00100 00 01 1100 1100 1100 0 10100 00 01 1100 1100 1100 0']
['10 01110 00 00 0000 0000 0000 0 11000 00 00 0000 0000 0000 0']
